<a href="https://colab.research.google.com/github/dantheman625/nlp_doc_info_extraction/blob/complete_pipe/complete_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
print("hello world")

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    AutoModelForSequenceClassification,
    pipeline
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification


# Datasets

Import Challenge data set


#

# Define models

# Baseline models
Define which model you used as a baseline model for the specific task

In [ ]:
baseline_ner_name = ""
baseline_re_name = "SpanBERT/spanbert-large-cased"


# Trained models

Define your trained model for the specific task

In [ ]:
trained_ner_name = "/path/to/your/ner/checkpoint"
trained_re_name = "/path/to/your/re/checkpoint"

# Model selection

Which model for NER, which for RE? -> Combination untrained/ untrained, trained/ trained, untrained/ trained, trained/ untrained

## Both baseline

In [ ]:
ner_model_name = baseline_ner_name
re_model_name  = baseline_re_name

## Both trained

In [ ]:
ner_model_name = trained_ner_name
re_model_name  = trained_re_name

## NER: trained, RE: baseline

In [ ]:
ner_model_name = trained_ner_name
re_model_name  = baseline_re_name

## NER: baseline, RE: trained

In [ ]:
ner_model_name = baseline_ner_name
re_model_name  = trained_re_name

# Load Models and Tokenizer

## NER

In [ ]:
ner_tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
ner_model     = AutoModelForTokenClassification.from_pretrained(ner_model_name)

## RE

In [ ]:
re_tokenizer  = AutoTokenizer.from_pretrained(re_model_name)
re_model      = AutoModelForSequenceClassification.from_pretrained(re_model_name)

# NER Eval

Output: Entity file -> content

# RE Eval

Input: Entity file, original challenge test file -> matching of entities to sentences (siehe wa) -> Liste mit dict